# Topic Modelling

Topic models assume that the words of each document arise from a mixture of **multinomials**. Across a collection, the documents share the same mixture components (called *topics*). Each document, however, is associated with its own mixture proportions (called *topic proprtions*). In this way, topic models represent documents heterogeneously - the documents share the same set of topics, but each exhibits them to a different degree. More generally, this is called *mixed membership*.

The central computational problem in topic modelling is posterior inference: Given a collection of documents, what are the topics that it exhibits and how does each document exhibit them? 

### Notations:
- Observations are *words*, organized into documents. The $n$th word in the $d$th document is $w_{dn}$. Each word is an element in a fixed vocabulary of $V$ terms.
- A *topic* $\beta_k$ is a distribution over the vocabulary. Each topic is a point on the $V−1$-simplex, a positive vector of length $V$ that sums to one. We denote the $w$th entry in the $k$th topic as $\beta_{kw}$. In LDA there are $K$ topics; in the HDP topic model there are an infinite number of topics.
- Each document in the collection is associated with a vector of *topic proportions* $\theta_d$, which is a distribution over topics. In LDA $\theta_d$ is a point on the $K − 1$-simplex. In the HDP topic model, $\theta_d$ is a point on the infinite simplex. (We give details about this below in Section 3.3.) We denote the $k$th entry of the topic proportion vector $\theta_d$ as $\theta_{dk}$.
- Each word in each document is assumed to have been drawn from a single topic. The *topic assignment* $z_{dn}$ indexes the topic from which $w_{dn}$ is drawn.

The only observed variables are the words of the documents. The topics, topic proportions, and topic assignments are latent variables

### Latent Dirichlet Allocation
![lda](figures/lda.png)
The graphical model representation of Latent Dirichlet allocation. Note that in practice each document $d$ may not have the same number of words $N$.

LDA is the simplest topic model. It assumes that each document exhibits $K$ topics with different proportions. The generative process is 
1. Draw topics $\beta_k\sim\mathrm{Dirichlet}(\eta,\ldots,\eta)$ for $k\in\{1,\ldots,K\}$.
2. For each document $d\in\{1,\ldots,D\}$:   
   1. Draw topic proportions $\theta \sim \mathrm{Dirichlet}(\alpha,\ldots,\alpha)$.
   2. For each word $w \in \{1,\ldots,N\}$:
       1. Draw topic assignment $z_{dn}\sim \mathrm{Multinomial}(\theta_d)$.
       2. Draw word $w_{dn}\sim \mathrm{Multinomial}(\beta_{z_{dn}})$.


| Var         | Type               | Conditional <img width=200/>                      | Param          | Relevant Expectations <img width=300/> |
|-------------|--------------------|-----------------------------------|----------------|-----------------------|
|$z_{dn}$ | Multinomial | ${\log\theta_{dk}+\log\beta_{k, w_{dn}}}$ | $\phi_{dn}$ | $\mathbb{E}[Z_{dn}^k] = \phi_{dn}^k$ |
|   |
|$\theta_d$ |Dirichlet | $\alpha+\sum_{n=1}^Nz_{dn}$ | $\gamma_{d}$ | $\mathbb{E}[\log\theta_{dk}] = \Psi(\gamma_{dk})-\sum_{j=1}^K\Psi(\gamma_{dj})$|
|
|$\beta_k$ | Dirichlet |$\eta + \sum_{d=1}^D\sum_{n=1}^Nz_{dn}^kw_{dn}$ |$\lambda_k$| $\mathbb{E}[\log\beta_{kv}] = \Psi(\lambda_{kv})-\sum_{y=1}^V\Psi(\lambda_{ky})$ |

In LDA, each document exhibits the same shared topics but with different proportions. LDA assumes Dirichlet priors for $\beta_k$ and $\theta_d$. Dirichlet distributions over the $D$-simplex takes $D+1$ parameters. 

LDA models an observed collection of documents $\mathbf{w} = w_{1:D}$, where each $w_d$ is a collection of words $w_{d,1:N}$. Analyzing documents amounts to posterior inference of $p(\beta,\theta,\mathbf{z}|\mathbf{w})$. Conditioned on the documents, the posterior distribution captures the topics that describe them ($\beta=\beta_{1:K}$), the degree to which each document exhibits those topics ($\theta=\theta_{1:D}$), and which topics each word was assigned to $\mathbf{z} = z_{1:D,1:N}$). We can use the posterior to explore large collections of documents.

The posterior is intractable to compute, so we need to approximate it. Here we derive stochastic inference for LDA.

### Complete conditionals and variational distributions
We specify the global and local variables of LDA to place it in the stochastic variational inference setting. In topic modelling, the local context is a document $d$. The local observations are its observed words $w_{d,1:N}$. The local hidden variables are the topic proportions $\theta_d$ and the topic assignments $z_{d,1:N}$. The global hidden variables are the topics $\beta_{1:K}$.

Remember that the complete conditional is the conditional distribution of a variable given all of the other variables, hidden and observed. In mean-field variational inference, the variational distributions of each variables are in the same family as the complete conditional.

We begin with the topic assignment $z_{dn}$. The complete conditional of the topic assignment is a multinomial,

$$
p(z_{dn}=k |\theta_d,\beta_{1:K},w_{dn})\propto \exp\{\log\theta_{dk} + \log\beta_{k,w_{dn}}\}.
$$

Thus its variational distribution is a multinomial $q(z_{dn}) = \mathrm{Multinomial}(\phi_{dn})$, where the variational parameter $\phi_{dn}$ is a point on the $K-1$-simplex. Per the mean-field approximation, each observed word is endowed with a different variational distribution for its topic assignment, allowing different words to be associated with different topics.

The complete conditional of the topic proportions is a Dirichlet,

$$
p(\theta_d|z_d) = \mathrm{Dirichlet}\left(\alpha + \sum_{n=1}^Nz_{dn}\right).
$$

Since $z_{dn}$ is an indicator vector, the $k$-th element of the parameter to this Dirichlet is the sum of the hyperparameter $\alpha$ and the number of words assigned to topic $k$ in document $d$. Note that, although we have assumed an exchangeable Dirichlet prior, when we condition on $z$ the conditional $p(\theta_d|z_d)$ is a non-exchangeable Dirichlet.

With this conditional, the variational distribution of the topic proportions is also $\mathrm{Dirichlet}q(\theta_d) = \mathrm{Dirichlet}(\gamma_d)$, where $\gamma_d$ is a $K$-vector Dirichlet parameter. There is a different variational Dirichlet parameter for each document, allowing different documents to be associated with different topics in different proportions.

These are local hidden variables. The complete conditionals only depend on other variables in the local context (i.e. the document) and the global variables; they do not depend on variables from other documents.

Finally, the complete conditional for the topic $\beta_k$ is also a Dirichlet

$$
p(\beta_k|\boldsymbol{z},\boldsymbol{w})=\mathrm{Dirichlet}\left(\eta + \sum_{d=1}^D\sum_{n=1}^Nz_{dn}^kw_{dn}\right).
$$

The $v$th element of the parameter to the Dirichlet conditional for topic $k$ is the sum of the hyperparameter $\eta$ and the number of times the term $v$ was assigned to topic $k$. This is a global variable - its complete conditional depends on the words and topic assignments of the entire collection.

The variational distribution for each topic is a $V$-dimensional Dirichlet,
$$
q(\beta_k) = \mathrm{Dirichlet}(\lambda_k)
$$

Now we have defined the complete conditionals we can derive the coordinate ascent variational inference algorithm.

#### Indicator vectors
We represent categorical variables like the topic assignments $z_{dn}$ and observed words $w_{dn}$ with *indicator vectors*. An indicator vector is a binary vector with a single one (also called one-hot-vectors) 

#### Dirichlet distribution.
A $K$-dimensional Dirichlet is a distribution on the $K-1$-simplex, i.e. positive vectors over $K$ elements that sum to one. It is parameterized by a positive $K$-vector $\gamma$,

$$
\mathrm{Dirichlet}(\theta; \gamma) = \frac{\Gamma\left(\sum_{i=1}^K\gamma_i\right)}{\prod_{i=1}^K\Gamma(\gamma_i)}\prod_{i=1}^K\theta^{\gamma_i-1}
$$
where $\Gamma(\cdot)$ is the Gamma function, which is a real-valued generalization of the factorial function.

The expectation of the Dirichlet is its normalized parameter

$$
\mathbb{E}[\theta_k|\gamma] = \frac{\gamma_k}{\sum_{i=1}^K\gamma_i}
$$

The expectation of its log uses $\Psi(\cdot)$, which is the first derivative of the log Gamma function,
$$
\mathbb{E}[\log\theta_k|\gamma] = \Psi(\gamma_k) - \Psi\left(\sum_{i=1}^K\gamma_i\right).
$$
This can be derived by putting the Dirichlet in exponential family form, noticing that $\log\theta$ is the vector of sufficient statistics and computing its expectation by taking the gradient of the log-normalizer with respect to the natural parameter vector $\gamma$.